In [1]:
#Import Dependencies
import pandas as pd
import numpy as np

#Import matplotlib dependencies
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress

#File to load
file_path = "../Data/pce_by_state_by_category.csv"

In [2]:
#Create dataframe
pce_df = pd.read_csv(file_path,encoding='cp1252') 
pce_df.sample(10)


,GeoFips,State,LineCode,Description,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
485,23000,Maine,6.0,Recreational goods and vehicles,1398.3,1455.4,1493.9,1568.3,1540.6,1481.0,1471.3,1502.4,1500.7,1500.5,1519.6,1532.2,1580.5,1651.9,1685.6,1785.0
1074,48000,Texas,19.0,Food services and accommodations,36166.4,39150.0,42621.5,45162.8,47704.9,48823.6,50962.8,54671.8,58438.6,61908.2,66550.3,71742.6,74505.8,78405.0,82922.6,86797.5
425,20000,Kansas,18.0,Recreation services,2821.0,2889.0,3123.5,3538.9,3535.7,3381.7,3215.5,3174.6,3250.6,3441.1,3615.3,3682.5,3770.8,3781.0,3832.4,3871.3
1397,97000,Rocky Mountain,6.0,Recreational goods and vehicles,10789.0,12033.4,12798.5,13483.6,12981.0,11533.9,11449.1,11703.4,12032.3,13049.6,13917.9,14539.1,15801.6,17484.3,18719.0,20351.9
175,9000,Connecticut,8.0,Nondurable goods,25669.7,27978.9,29593.8,30801.2,31526.3,29865.7,30809.1,32349.7,32747.5,33668.2,34837.0,34849.6,35011.8,36378.0,37032.3,37378.1
1278,92000,Mideast,7.0,Other durable goods,23779.1,24964.9,26931.9,28378.9,27920.2,26903.2,27982.2,30025.5,30548.9,31977.1,32987.7,33499.2,33797.4,34340.8,35731.4,36834.0
1018,46000,South Dakota,11.0,Gasoline and other energy goods,1017.0,1232.5,1408.2,1522.3,1734.5,1280.5,1508.9,1827.0,1877.0,1894.5,1853.5,1434.4,1276.8,1414.2,1537.2,1447.0
524,24000,Maryland,21.0,Other services,16611.7,17872.2,19265.2,20173.4,20926.7,21095.9,21526.8,21770.5,21870.7,22005.1,22922.9,23434.7,24409.9,25081.2,26020.7,27036.4
659,30000,Montana,12.0,Other nondurable goods,1769.9,1876.7,2081.8,2258.9,2355.6,2416.2,2483.0,2546.4,2622.9,2678.6,2789.9,2933.0,3004.6,3077.8,3216.6,3395.3
760,34000,New Jersey,17.0,Transportation services,12870.7,13299.8,13927.9,13866.1,13741.0,13249.8,13176.2,14238.6,14594.1,15556.0,16232.5,16553.1,17040.9,17441.7,18172.0,18373.8


In [3]:
# Show column names 
for col in pce_df.columns: 
    print(col) 

GeoFips
State
LineCode
Description
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [4]:
#Load in Latlngs
latlong_file_path = "../Data/statelatlong.csv"
latlong_df = pd.read_csv(latlong_file_path,encoding='cp1252') 
latlong_df = latlong_df.drop('State', 1)
latlong_df = latlong_df.rename(columns={'City': 'State'})
latlong_df.head()

,Latitude,Longitude,State
0,32.601011,-86.680736,Alabama
1,61.302501,-158.775020,Alaska
2,34.168219,-111.930907,Arizona
3,34.751928,-92.131378,Arkansas
4,37.271875,-119.270415,California


In [5]:
pce_with_latlng = pd.merge(pce_df, latlong_df, on ='State', how ='left')

pce_with_latlng.head() 

,GeoFips,State,LineCode,Description,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,Latitude,Longitude
0,0,United States,1.0,Personal consumption expenditures,8206825.4,8741177.6,9253947.4,9699133.5,9968412.9,9833212.0,...,10997927.0,11309307.3,11814798.4,12290166.3,12762930.6,13333393.5,13982899.0,14533254.0,NaN,NaN
1,0,United States,2.0,Goods,2896184.3,3076982.5,3233258.1,3359733.4,3355303.7,3171025.7,...,3628852.0,3722069.7,3855001.8,3915678.3,3991320.6,4165294.4,4361471.7,4500860.9,NaN,NaN
2,0,United States,3.0,Durable goods,1080574.3,1128610.2,1158333.1,1188043.0,1098760.6,1012121.3,...,1144221.0,1189413.3,1242075.5,1307605.2,1350222.4,1410678.3,1481551.7,1534351.3,NaN,NaN
3,0,United States,4.0,Motor vehicles and parts,409310.9,409954.2,394949.3,400573.0,343337.2,318583.1,...,396607.9,417533.0,441992.0,475346.0,485553.6,503642.6,523226.5,521845.9,NaN,NaN
4,0,United States,5.0,Furnishings and durable household equipment,254310.4,271273.5,283597.6,283457.6,264348.6,238275.2,...,253938.0,263589.7,276166.5,294219.7,309367.1,324744.9,343326.3,357440.3,NaN,NaN
